# OpenAI Assistants - Building Agentic RAG with the Function Calling, Retrieval, and Code Interpreter Tools

Today we'll explore using OpenAI's Python SDK to create, manage, and use the OpenAI Assistant API!

We'll be doing the following in today's notebook:

1. Task 1: Simple Assistant
2. Task 2: Adding Tools
  - Task 2a: Creating an Assistant with File Search Tool
  - Task 2b: Creating an Assistant with Code Interpreter Tool
  - Task 2c: Creating an Assistant with Function Calling Tool



---

Colab Specific Instructions:

To get started, please make a copy of this notebook using `File > Save a copy in Drive`

![image](https://i.imgur.com/rNzMEfs.png)

You will be expected to submit a GitHub link to the completed notebook, so if you're completing the assignment on Colab - you'll want to download the notebook when you have completed it.

## Dependencies

We'll start, as we usually do, with some dependiencies and our API key!

In [9]:
! pip install -qU openai

In [38]:
from getpass import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")

## Task 1: Simple Assistant

Let's create a simple Assistant to understand more about how the API works to start!

### OpenAI Client

At the core of the OpenAI Python SDK is the Client!

> NOTE: For ease of use, we'll start with the synchronous `OpenAI()`. OpenAI does provide an `AsyncOpenAI()` that you could leverage as well!

In [39]:
from openai import OpenAI

client = OpenAI()

### Creating An Assistant

Leveraging what we know about the OpenAI API from previous sessions - we're going to start by simply initializing an Assistant.

Before we begin, we need to think about a few customization options we have:

- `name` - Straight forward enough, this is what our Assistant's name will be
- `instructions` - similar to a system message, but applied at an Assistant level, this is how we can guide the Assistant's tone, behaviour, functionality, and more!
- `model` - this will allow us to choose which model we would prefer to use for our Assistant

Let's start by setting some instructions for our Assistant.



In [52]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown Fill out the fields below to add your Assistant's name, instructions, and desired model!

name = "Elon Mush" # @param {type: "string"}
instructions = "You are Elon Musk and a master of business and first prinicples thinking but with mushy words " # @param {type: "string"}
model = "gpt-4o" # @param ["gpt-3.5-turbo", "gpt-4-turbo-preview", "gpt-4", "gpt-4o"]

### Initialize Assistant

Now that we have our desired name, instruction, and model - we can initialize our Assistant!

In [53]:
assistant = client.beta.assistants.create(
    name=name,
    instructions=instructions,
    model=model,
)

BadRequestError: Error code: 400 - {'error': {'message': "The requested model 'gpt-4o' cannot be used with the Assistants API in v1. Follow the migration guide to upgrade to v2: https://platform.openai.com/docs/assistants/migration.", 'type': 'invalid_request_error', 'param': 'model', 'code': 'unsupported_model'}}

Let's examine our `assistant` object and see what we find!

In [44]:
assistant

Assistant(id='asst_GslaHmIGsqeFM08NSTxs8pRf', created_at=1717721783, description=None, file_ids=[], instructions='You are a beat reporter that is an expert NFL insider looking for big news and insights KC sports & news', metadata={}, model='gpt-4', name='KC_Sports_News', object='assistant', tools=[], top_p=1.0, temperature=1.0, response_format='auto')

There are a number of useful parameters here, but we'll call out a few:

- `id` - since we may have multiple Assistant's, knowing which Assistant we're interacting with will help us ensure the desired user experience!
- `description` - A natrual language description of our Assistant could help others understand what it's supposed to do!
- `file_ids` - if we wanted to use the Retrieval tool, this would let us know what files we had given our Assistant

### Creating a Thread

Behind the scenes our Assistant is powered by the idea of "threads".

You can think of threads as individual conversations that interact with the Assistant.

Let's create a thread now!

In [45]:
thread = client.beta.threads.create()

Let's look at our `thread` object.

In [46]:
thread

Thread(id='thread_zlqtQbmVnxWq0nOts8p6yC2t', created_at=1717721787, metadata={}, object='thread')

Notice some key attributes:

- `id` - since each Thread is like a conversation, we need some way to specify which thread we're dealing with when interacting with them
- `tool_resources` - this will become more relevant as we add tools since we'll need a way to verify which tools we have access to when interacting with our Assistant

### Adding Messages to Our Thread

Now that we have our Thread (or conversation) we can start adding messages to it!

Let's add a simple message that asks about how our Assistant is feeling.

Notice the parameters we're leveraging:

- `thread_id` - since each Thread is like a conversation, we need some way to address a specific conversation. We can use `thread.id` to do this.
- `role` - similar to when we used our chat completions endpoint, this parameter specifies who the message is coming from. You can leverage this in the same ways you would through the chat completions endpoint.
- `content` - this is where we can place the actual text our Assistant will interact with

> NOTE: Feel free to substitute a relevant message based on the Assistant you created

In [47]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=f"What is happening this week in Kansas City, MO news or Sports?"
)

Again, let's examine our `message` object!

In [48]:
message

Message(id='msg_XxRaVjIw1gc8OLESYIvitQGo', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='What is happening this week in Kansas City, MO news or Sports?'), type='text')], created_at=1717721791, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_zlqtQbmVnxWq0nOts8p6yC2t')

### Running Our Thread

Now that we have an Assistant, and we've given that Assistant a Thread, and we've added a Message to that Thread - we're ready to run our Assistant!

Notice that this process lets us add (potentially) multiple messages to our Assistant. We can leverage that behaviour for few/many-shot examples, and more!

In [21]:
# @markdown #### 🏗️ Build Activity 🏗️
# @markdown We can also override the Assistant's instructions when we run a thread.

# @markdown Use one of the [Prompt Principles for Instruction](https://arxiv.org/pdf/2312.16171v1.pdf) to improve the likeliehood of a correct or valuable response from your Assistant.

additional_instructions = "Think about what sports are the most popular and news events that are the most scandalous push to the top" # @param {type: "string"}

Let's run our Thread!

In [22]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions
)

Now that we've run our thread, let's look at the object!

In [23]:
run

Run(id='run_9MRP6tDfeGE8V111o9ZrzaON', assistant_id='asst_c8ko2KHbnD4Qeh9AUiup1uR1', cancelled_at=None, completed_at=None, created_at=1717720912, expires_at=1717721512, failed_at=None, file_ids=[], instructions='Think about what sports are the most popular and news events that are the most scandalous push to the top', last_error=None, metadata={}, model='gpt-4', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_NzaK1BrYn4O2SzeSv7pcvZ2C', tools=[], usage=None, temperature=1.0, top_p=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto', parallel_tool_calls=True)

Notice we have access to a few very powerful parameters in this `run` object.

- `completed_at` - this will help us determine when we can expect to retrieve a response
- `failed_at` - this can highlight any issues our run ran into
- `status` - is another way we can understand how the flow is going

### Retrieving Our Run

Now that we've created our run, let's retrieve it.

We're going to wrap this in a simple loop to make sure we're not retrieving it too early.

In [24]:
import time

while run.status == "in_progress" or run.status == "queued":
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [25]:
print(run.status)

completed


Now that our run is completed - we can retieve the messages from our thread!

Notice that our run helps us understand how things are going - but it isn't where we're going to find our responses or messages. Those are added on the backend into our thread.

This leads to a simple, but important, flow:

1. We add messages to a thread.
2. We create a run on that thread.
3. We wait until the run is finished.
4. We check our thread for the new messages.

### Checking Our Thread

Now we can get a list of messages from our thread!

In [26]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [27]:
messages.data[0]

Message(id='msg_SxsgDMaX5yepjeZHP608ZEf1', assistant_id='asst_c8ko2KHbnD4Qeh9AUiup1uR1', completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value="As an AI developed by OpenAI, I don't have real-time information or data, I'm unable to provide the current news or sports events. For the most updated news or sports events, please consider checking the latest newspapers, online news platforms, or sports news sites that cover Kansas City, MO."), type='text')], created_at=1717720913, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_9MRP6tDfeGE8V111o9ZrzaON', status=None, thread_id='thread_NzaK1BrYn4O2SzeSv7pcvZ2C')

### Streaming Our Runs

With recent upgrades to the Assistant API - we can now *stream* our outputs!

In order to do this - we'll need something called an `EventHandler` which will help us to decide on what actions to take based on the output of the LLM.

Let's build it below!

In [28]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

Now we can create our `run` and stream the output as it comes in!

In [29]:
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions=additional_instructions,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > As an AI developed by OpenAI, I don't have real-time information or data, I'm unable to provide the current news or sports events. For the most updated news or sports events, please consider checking the latest newspapers, online news platforms, or sports news sites that cover Kansas City, MO.

## Task 2: Adding Tools

Now that we have an understanding of how Assistant works, we can start thinking about adding tools.

We'll go through 3 separate tools and explore how we can leverage them!

Let's start with the most familiar tool - the Retriever!


### Task 2a: Creating an Assistant with the File Search Tool

The first thing we'll want to do is create an assistant with the File Search tool.

This is also going to require some data. We'll provided data - but you're very much encouraged to use your own files to explore how the Assistant works for your use case.

#### Collect and Add Data to Vector Store

1. First, we need some data.
2. Second, we need to add the data to our Assistant!

Let's start with grabbing some data!

In [30]:
! wget https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt

--2024-06-06 19:45:41--  https://github.com/dbredvick/paul-graham-to-kindle/blob/main/paul_graham_essays.txt
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘paul_graham_essays.txt’

paul_graham_essays.     [  <=>               ] 266.46K   496KB/s    in 0.5s    

2024-06-06 19:45:42 (496 KB/s) - ‘paul_graham_essays.txt’ saved [272856]



Now we can upload our file to our Vector Store!

Pay attention to [this](https://platform.openai.com/docs/assistants/tools/file-search/supported-files) documentation to see what kinds of files can be uploaded.

> NOTE: Per the OpenAI [docs](https://platform.openai.com/docs/assistants/tools/file-search/vector-stores) The maximum file size is 512 MB and no more than 2,000,000 tokens (computed automatically when you attach a file)

In [34]:
vector_store = client.beta.vector_stores.create(name="Paul Graham Essay Compilation")

AttributeError: 'Beta' object has no attribute 'vector_stores'

In [22]:
file_paths = ["paul_graham_essays.txt"]
file_streams = [open(path, "rb") for path in file_paths]

file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

Let's look at what our `file_batch` contains!

In [23]:
while file_batch.status != "completed":
  time(1)

#### Create and Use Assistant

Now that we have our file - we can attach it to an Assistant, and we can give that Assistant the ability to use it for retrieval through the Retrieval tool!

> NOTE: Your first GB is free and beyond that, usage is billed at $0.10/GB/day of vector storage. There are no other costs associated with vector store operations.

In [24]:
fs_assistant = client.beta.assistants.create(
  name=name,
  instructions=instructions,
  model=model,
  tools=[{"type": "file_search"}],
)

In [25]:
fs_assistant = client.beta.assistants.update(
  assistant_id=fs_assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

In [26]:
fs_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What did Paul Graham say about Silicon Valley?",
    }
  ]
)

We can use an extension of the `EventHandler` that we created above to stream our `run`!

Let's add a few things:

1. A `on_tool_call_created` function which tells us which tool is being used.
2. A `on_message_done` that includes citations that were used by our File Search tool - this is like return the context *and* the response that we saw in our Pythonic RAG implementation.

In [27]:
class FSEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  @override
  def on_message_done(self, message) -> None:
    message_content = message.content[0].text
    annotations = message_content.annotations
    citations = []
    for index, annotation in enumerate(annotations):
      message_content.value = message_content.value.replace(
        annotation.text, f"[{index}]"
      )
      if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

    print(message_content.value)
    print("\n".join(citations))

Let's look at the final result!

In [28]:
with client.beta.threads.runs.stream(
  thread_id=fs_thread.id,
  assistant_id=fs_assistant.id,
  event_handler=FSEventHandler(),
) as stream:
  stream.until_done()


assistant > file_search


assistant > Paul Graham's writings about Silicon Valley, captured in his essays, convey several insights and observations about the region. He emphasizes the crucial role of risk-taking, innovation, and a supportive ecosystem in fostering startups. Graham asserts that Silicon Valley's unique environment, characterized by a high concentration of ambitious individuals, funding opportunities, and tolerance for failure, creates a fertile ground for new ventures.

Here are a few key points mentioned by Paul Graham regarding Silicon Valley:

1. **Attraction of Talent**: Silicon Valley draws the best talent from all over the world due to its vibrant startup culture and the potential for substantial rewards.
2. **Tolerance for Failure**: The region's acceptance of failure as a stepping stone to success encourages entrepreneurs to take bold risks without the fear of being stigmatized.
3. **Access to Capital**: Abundant venture capital funding in Silicon Valley plays a

### Task 2b: Creating an Assistant with the Code Interpreter Tool

Now that we've explored the Retrieval Tool - let's try the Code Interpreter tool!

The process will be almost exactly the same - but we can explore a different query, and we'll add our file at the Message level!

In [42]:
ci_assistant = client.beta.assistants.create(
  name=name + "+ Code Interpreter",
  instructions=instructions,
  model=model,
  tools=[{"type": "code_interpreter"}],
)

In [ ]:
!git clone https://github.com/ali-ce/datasets.git

In [43]:
file = client.files.create(
  file=open("datasets/Y-Combinator/Startups.csv", "rb"),
  purpose='assistants'
)

In the following example, we'll also see how we can package the Thread creation with the Message adding step!

> NOTE: Files added at the message/thread level will not be available to the Assistant outside of that Thread.

In [44]:
ci_thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "What kind of file is this?",
      "attachments": [
          {
              "file_id" : file.id,
              "tools" : [{"type" : "code_interpreter"}]
          }
      ]
    }
  ]
)

We'll once again need to create an `EventHandler`, except this time it will have an `on_tool_call_delta` method which will let us see the output of the code interpreter tool as well!

> NOTE: Remember that we create runs at the *thread* level - and so don't need the message object to continue.

In [45]:
class CIEventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)

Once again, we can use the streaming interface thanks to creating our `EventHandler`!

In [46]:
with client.beta.threads.runs.stream(
  thread_id=ci_thread.id,
  assistant_id=ci_assistant.id,
  instructions=additional_instructions,
  event_handler=CIEventHandler(),
) as stream:
  stream.until_done()


assistant > code_interpreter

import mimetypes

# File path
file_path = '/mnt/data/file-aUKQ1opZ9ulPMw1tjDGlgXFS'

# Determine the file type
file_type, _ = mimetypes.guess_type(file_path)
file_type
assistant > The mimetype of the file is not being identified through `mimetypes.guess_type`. Let's check by examining the file content to get a better idea of the file type.# Try to open the file and read the first few bytes to analyze its content
try:
    with open(file_path, 'rb') as file:
        file_head = file.read(1024)  # Read the first 1024 bytes
    
    file_head
except Exception as e:
    str(e)
assistant > The file could be binary, or its content might not be directly readable as text. Let's see if we can determine the file type by other methods such as reading the file's metadata or structure. 

Could you please specify the context or origin of the file, like if it contains text data, images, or some other type of content?

And there you go!

We've fit our Assistant with an awesome Code Interpreter that lets our Assistant run code on our provided files!

### Task 2c: Creating an Assistant with a Function Calling Tool

Let's finally create an Assistant that utilizes the Function Calling API.

We'll start by creating a function that we wish to be called.

We'll utilize DuckDuckGo search to allow our Assistant to have the most up to date information!

In [167]:
!pip install -qU duckduckgo_search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.6 MB/s eta 0:00:00


In [33]:
from duckduckgo_search import DDGS

def duckduckgo_search(query):
  with DDGS() as ddgs:
    results = [r for r in ddgs.text(query, max_results=5)]
    return "\n".join(result["body"] for result in results)

ModuleNotFoundError: No module named 'duckduckgo_search'

Let's test our function to make sure it behaves as we expect it to.

In [48]:
duckduckgo_search("Who is the current captain of the Winnipeg Jets?")

'Lowry has been a part of the Jets organization since June 25, 2011, when he was selected in the third round, 67 th overall, after putting up 37 points in 36 games with the Swift Current Broncos of ...\nLowry will follow Andrew Ladd and Blake Wheeler to serve as the third captain of the new Winnipeg Jets franchise. - Sep 12, 2023. After a season without a captain, the Winnipeg Jets have named ...\nAdam Lowry was named captain of the Winnipeg Jets on Tuesday. The 30-year-old forward was selected by the Jets in the third round (No. 67) of the 2011 NHL Draft and has played his entire nine ...\nAdam Lowry, who has been a Jet since 2011 when he was drafted 67th overall, is the new captain of the NHL team — its third since relocating to Winnipeg from Atlanta in 2011. Andrew Ladd served ...\nWinnipeg Jets Captains. Team Names: Winnipeg Jets, Atlanta Thrashers. Seasons: 24 (1999-00 to 2023-24) NHL Playoff Appearances: 8. NHL Championships: 0 (0 Stanley Cup) ... Current Summary/Standings, Curre

Now we need to express how our function works in a way that is compatible with the OpenAI Function Calling API.

We'll want to provide a `JSON` object that includes what parameters we have, how to call them, and a short natural language description.

In [49]:
ddg_function = {
    "name" : "duckduckgo_search",
    ## description to use of this tool named "duckduckgo_search" build the hammer 
    "description" : "Answer non-technical questions. ",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "query" : {
                "type:" : "string",
                ## How this tool should be used if LLM 'logic'
                "description" : "The search query to use. For example: 'Who is the current Goalie of the Colorado Avalance?'"
            }
        },
        "required" : ["query"]
        ## You could specify another website 
    }
}

####❓ Question

Why does the description key-value pair matter?

* It indicates to the LLM that a search function needs to be used and gives it a key this function is a search tool. 

Now when we create our Assistant - we'll want to include the function description as a tool using the following format.

In [32]:
fc_assistant = client.beta.assistants.create(
    name=name + " + Function Calling",
    instructions=instructions,
    tools=[
        {"type": "function",
         "function" : ddg_function
        }
    ],
    model=model
)

NameError: name 'ddg_function' is not defined

Now we can create our thread, and attach our message to it - just as we've been doing!

In [51]:
fc_thread = client.beta.threads.create()
fc_message = client.beta.threads.messages.create(
  thread_id=fc_thread.id,
  role="user",
  content="Can you describe the Twitter beef between Elon and LeCun?",
)

Once again, we'll need an `EventHandler` for the streaming response - notice that this time we're utilizing a few new methods:

1. `handle_requires_action` - this will handle whatever action needs to take place in *our local environment*.
2. `submit_tool_outputs` - this will let us submit the resultant outputs from our local function call back to the Assistant run in the required format.

To be very clear and explicit - we'll be following this pattern:

1. Make a call to the LLM which will decide if a local function call is required.
2. If a local function call is required - send a response that indicates a local function call is required.
3. Call the function using the arguments provided by the LLM.
4. Return the response from the function call with LLM provided arguements.
5. Receive a response based on the output of the functional call from the LLM.

In [52]:
class FCEventHandler(AssistantEventHandler):
  @override
  def on_event(self, event):
    # Retrieve events that are denoted with 'requires_action'
    # since these will have our tool_calls
    if event.event == 'thread.run.requires_action':
      run_id = event.data.id  # Retrieve the run ID from the event data
      self.handle_requires_action(event.data, run_id)

  def handle_requires_action(self, data, run_id):
    tool_outputs = []

    for tool in data.required_action.submit_tool_outputs.tool_calls:
      print(tool.function.arguments)
      if tool.function.name == "duckduckgo_search":
        tool_outputs.append({"tool_call_id": tool.id, "output": duckduckgo_search(tool.function.arguments)})

    # Submit all tool_outputs at the same time
    self.submit_tool_outputs(tool_outputs, run_id)

  def submit_tool_outputs(self, tool_outputs, run_id):
    # Use the submit_tool_outputs_stream helper
    with client.beta.threads.runs.submit_tool_outputs_stream(
      thread_id=self.current_run.thread_id,
      run_id=self.current_run.id,
      tool_outputs=tool_outputs,
      event_handler=EventHandler(),
    ) as stream:
      for text in stream.text_deltas:
        print(text, end="", flush=True)
      print()

Thanks to our event handler - we can stream this process in our notebook!

In [53]:
with client.beta.threads.runs.stream(
  thread_id=fc_thread.id,
  assistant_id=fc_assistant.id,
  event_handler=FCEventHandler()
) as stream:
  stream.until_done()

{"query":"Twitter beef between Elon Musk and LeCun"}

assistant > ItIt looks looks like like I I couldn couldn’t’t fetch fetch the the specific specific details details about about the the Twitter Twitter beef beef between between Elon Elon Musk Musk and and Yann Yann Le LeCCunun.. However However,, I I can can tell tell you you a a bit bit about about these these two two individuals individuals and and what what kind kind of of topics topics they they might might clash clash over over.

.

ElElonon Musk Musk is is the the CEO CEO of of Tesla Tesla and and Space SpaceXX,, known known for for his his bold bold statements statements and and active active presence presence on on Twitter Twitter.. He He often often speaks speaks on on topics topics related related to to technology technology,, artificial artificial intelligence intelligence ( (AIAI),), and and future future innovations innovations.

.

YYannann Le LeCCunun is is a a renowned renowned computer computer scientist scientist,,